In [5]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load preprocessed train and test datasets
train_df = pd.read_csv("/kaggle/input/opt-dataset/final optimised dataset/train_dataset (1).csv")
test_df = pd.read_csv("/kaggle/input/opt-dataset/final optimised dataset/test.csv")
test_labels_df = pd.read_csv("/kaggle/input/opt-dataset/final optimised dataset/test_labels.csv")

# Merge test set with labels
test_df = test_df.merge(test_labels_df, on="id")

# Identify label columns (assuming they start from 2nd column in test_labels_df)
label_columns = list(test_labels_df.columns[1:])  # Excluding 'id' column

# Convert pandas to Hugging Face dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# Load Deberta tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# First, check if the column exists and what type of data it contains
print(test_ds.column_names)  # Check if "comment_text" exists
print(type(test_ds["comment_text"][0]) if "comment_text" in test_ds.column_names else "Column not found")

# Modified tokenization function with error handling
def tokenize_function(examples):
    # Filter out None/NaN values and ensure all inputs are strings
    texts = []
    for text in examples["comment_text"]:
        if text is None:
            texts.append("")  # Replace None with empty string
        elif not isinstance(text, str):
            texts.append(str(text))  # Convert non-strings to strings
        else:
            texts.append(text)
    
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="np"  # Use numpy arrays instead of PyTorch tensors for now
    )

# Try processing in smaller batches
train_ds = train_ds.map(tokenize_function, batched=True, batch_size=64)
test_ds = test_ds.map(tokenize_function, batched=True, batch_size=64)

# Ensure labels are structured as a list of floats
def format_labels(example):
    example["labels"] = [float(example[col]) for col in label_columns]  # Convert labels to float
    return example

train_ds = train_ds.map(format_labels)
test_ds = test_ds.map(format_labels)

# Remove unnecessary columns (text, id, and individual label columns after structuring)
columns_to_remove = ["comment_text", "id"] + label_columns
train_ds = train_ds.remove_columns(columns_to_remove)
test_ds = test_ds.remove_columns(columns_to_remove)

# Ensure dataset format remains list of floats
def ensure_list_format(example):
    example["labels"] = [float(x) for x in example["labels"]]  # Ensure list of floats
    return example

train_ds = train_ds.map(ensure_list_format)
test_ds = test_ds.map(ensure_list_format)

print("✅ Deberta Dataset loaded, tokenized & formatted successfully!")

# Save datasets (Optional: Avoid re-running preprocessing)
train_ds.save_to_disk("/kaggle/working/train_ds_deberta")
test_ds.save_to_disk("/kaggle/working/test_ds_deberta")

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


['id', 'comment_text', 'severely_toxic', 'moderately_toxic', 'non_toxic']
<class 'str'>


Map:   0%|          | 0/298513 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/298513 [00:00<?, ? examples/s]

Map:   0%|          | 0/127936 [00:00<?, ? examples/s]

✅ Deberta Dataset loaded, tokenized & formatted successfully!


Saving the dataset (0/1 shards):   0%|          | 0/298513 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/127936 [00:00<?, ? examples/s]

In [6]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, AdamW
import shutil
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

# Load DeBERTa Base model with sigmoid outputs for multi-label classification
deberta_model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base", 
    num_labels=3,  # Matches one-hot encoding format
    problem_type="multi_label_classification"
)

# Increase dropout for better regularization
deberta_model.config.hidden_dropout_prob = 0.3
deberta_model.config.attention_probs_dropout_prob = 0.3

# Freeze Transformer Layers, Train Only Classifier
for param in deberta_model.deberta.parameters():
    param.requires_grad = False  
for param in deberta_model.classifier.parameters():
    param.requires_grad = True  

# Hybrid Loss (Focal + BCE With Logits)
class HybridLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, ce_weight=0.5):
        super(HybridLoss, self).__init__()
        self.focal = FocalLoss(alpha, gamma)
        self.bce = nn.BCEWithLogitsLoss()
        self.ce_weight = ce_weight
        
    def forward(self, inputs, targets):
        return self.ce_weight * self.bce(inputs, targets.float()) + (1 - self.ce_weight) * self.focal(inputs, targets)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        
    def forward(self, inputs, targets):
        BCE_loss = self.bce(inputs, targets.float())
        probs = torch.sigmoid(inputs)
        pt = torch.where(targets.bool(), probs, 1-probs)
        alpha_factor = torch.where(targets.bool(), self.alpha, 1 - self.alpha)
        focal_weight = alpha_factor * (1 - pt).pow(self.gamma)
        return (focal_weight * BCE_loss).mean()

# Training arguments with auto-save best model - using epochs instead of steps
training_args = TrainingArguments(
    output_dir="./deberta_model",
    evaluation_strategy="epoch",      # Evaluation once per epoch
    save_strategy="epoch",            # Save once per epoch
    save_total_limit=1,
    logging_strategy="epoch",         # Log once per epoch
    learning_rate=8e-6,               # Slightly higher LR for base model compared to large
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    per_device_train_batch_size=8,    # Can use larger batch size with smaller model
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.1,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_auc",
    greater_is_better=True,
    max_grad_norm=1.0,
    label_smoothing_factor=0.05,
    fp16=True,
    gradient_accumulation_steps=1,    # Can reduce or eliminate with smaller model
    seed=42,
    data_seed=42,
)

# Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    sigmoid_logits = torch.sigmoid(torch.tensor(logits)).numpy()
    predictions = (sigmoid_logits > 0.5).astype(np.int8)
    
    accuracy = accuracy_score(labels, predictions)
    
    try:
        f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
        f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
        auc_scores = [roc_auc_score(labels[:, i], sigmoid_logits[:, i]) for i in range(labels.shape[1]) if len(np.unique(labels[:, i])) > 1]
        mean_auc = np.mean(auc_scores) if auc_scores else 0.0
    except Exception as e:
        print(f"Error in metric calculation: {e}")
        f1_micro, f1_macro, mean_auc = 0, 0, 0
    
    return {"accuracy": accuracy, "f1_micro": f1_micro, "f1_macro": f1_macro, "auc": mean_auc}

# Custom Trainer with Hybrid Loss
class HybridLossTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = HybridLoss(alpha=0.5, gamma=2.0, ce_weight=0.5)
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  
        labels = inputs["labels"].float()  # Keep labels in one-hot format
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Define Trainer
trainer = HybridLossTrainer(
    model=deberta_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

# Step 1: Train Only Classifier
print("🚀 Training classifier only...")
trainer.train()

# Step 2: Unfreeze last 4 layers EXACTLY
print("🔓 Unfreezing last 4 layers for further fine-tuning...")
for layer in deberta_model.deberta.encoder.layer[-4:]:
    for param in layer.parameters():
        param.requires_grad = True

# Apply Layer-wise Learning Rate
optimizer = AdamW([
    {"params": [p for n, p in deberta_model.deberta.named_parameters() if "encoder.layer" not in n or 
               (int(n.split("encoder.layer.")[1].split(".")[0]) < len(deberta_model.deberta.encoder.layer) - 4)], "lr": 2e-6},
    {"params": [p for n, p in deberta_model.deberta.named_parameters() if "encoder.layer" in n and 
               (int(n.split("encoder.layer.")[1].split(".")[0]) >= len(deberta_model.deberta.encoder.layer) - 4)], "lr": 5e-6},
    {"params": deberta_model.classifier.parameters(), "lr": 8e-6},
], weight_decay=0.1)

trainer.args.num_train_epochs = 2  
trainer.optimizer = optimizer  

# Step 2: Fine-tune last 4 layers
print("🚀 Fine-tuning last 4 layers...")
trainer.train()

# Save best model
best_model_path = "./best_deberta_base"
trainer.save_model(best_model_path)
tokenizer.save_pretrained(best_model_path)
shutil.make_archive(best_model_path, 'zip', best_model_path)
print("✅ Best Model Saved!")

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-1fd5eaa0ecdf>:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `HybridLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


🚀 Training classifier only...


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Auc
1,0.273800,0.211697,0.620717,0.723639,0.707308,0.906977
2,0.217300,0.200083,0.652983,0.750856,0.737145,0.916075


...

🔓 Unfreezing last 4 layers for further fine-tuning...
🚀 Fine-tuning last 4 layers...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,Auc
1,0.120900,0.137014,0.969883,0.971527,0.971638,0.997113
2,0.129100,0.134712,0.973455,0.974453,0.974503,0.997372


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Best Model Saved!
